In [39]:
# Basic Libraries
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

# Metrics
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# ML Models
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [21]:
filepath_csv = "C:/Users/prabh/Desktop/Python/Datasets/Air Quality/AirQualityUCI.csv"
df = pd.read_excel(filepath_xlsx)
display(df.head())

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,2004-03-10,18:00:00,2.6,1360.00,150,11.881723,1045.50,166.0,1056.25,113.0,1692.00,1267.50,13.60,48.875001,0.757754
1,2004-03-10,19:00:00,2.0,1292.25,112,9.397165,954.75,103.0,1173.75,92.0,1558.75,972.25,13.30,47.700000,0.725487
2,2004-03-10,20:00:00,2.2,1402.00,88,8.997817,939.25,131.0,1140.00,114.0,1554.50,1074.00,11.90,53.975000,0.750239
3,2004-03-10,21:00:00,2.2,1375.50,80,9.228796,948.25,172.0,1092.00,122.0,1583.75,1203.25,11.00,60.000000,0.786713
4,2004-03-10,22:00:00,1.6,1272.25,51,6.518224,835.50,131.0,1205.00,116.0,1490.00,1110.00,11.15,59.575001,0.788794


In [22]:
print(df.columns)

Index(['Date', 'Time', 'CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)',
       'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)',
       'PT08.S5(O3)', 'T', 'RH', 'AH'],
      dtype='object')


In [23]:
unwanted_features = ['Time', 'PT08.S1(CO)', 'PT08.S2(NMHC)', 'PT08.S3(NOx)', 'PT08.S4(NO2)', 'PT08.S5(O3)']

In [25]:
# # RUN ONLY ONCE! DO NOT RERUN/EDIT

# df = df.drop(unwanted_features, axis=1)

In [26]:
display(df.head())

,Date,CO(GT),NMHC(GT),C6H6(GT),NOx(GT),NO2(GT),T,RH,AH
0,2004-03-10,2.6,150,11.881723,166.0,113.0,13.60,48.875001,0.757754
1,2004-03-10,2.0,112,9.397165,103.0,92.0,13.30,47.700000,0.725487
2,2004-03-10,2.2,88,8.997817,131.0,114.0,11.90,53.975000,0.750239
3,2004-03-10,2.2,80,9.228796,172.0,122.0,11.00,60.000000,0.786713
4,2004-03-10,1.6,51,6.518224,131.0,116.0,11.15,59.575001,0.788794


0. Date (DD/MM/YYYY)
1. Time (HH.MM.SS)
2. True hourly averaged concentration CO in mg/m^3 (reference analyzer)
3. PT08.S1 (tin oxide) hourly averaged sensor response (nominally CO targeted)
4. True hourly averaged overall Non Metanic HydroCarbons concentration in microg/m^3 (reference analyzer)
5. True hourly averaged Benzene concentration in microg/m^3 (reference analyzer)
6. PT08.S2 (titania) hourly averaged sensor response (nominally NMHC targeted)
7. True hourly averaged NOx concentration in ppb (reference analyzer)
8. PT08.S3 (tungsten oxide) hourly averaged sensor response (nominally NOx targeted)
9. True hourly averaged NO2 concentration in microg/m^3 (reference analyzer)
10. PT08.S4 (tungsten oxide) hourly averaged sensor response (nominally NO2 targeted)
11. PT08.S5 (indium oxide) hourly averaged sensor response (nominally O3 targeted)
12. Temperature in Â°C
13. Relative Humidity (%)
14. AH Absolute Humidity

In [27]:
# At the time of creating this project, which was somewhere around late 2019, I had no experience on dealing with date-time
# Therefore, the date column is all together dropped

df = df.drop(['Date'], axis=1)

In [28]:
print(df.columns)

Index(['CO(GT)', 'NMHC(GT)', 'C6H6(GT)', 'NOx(GT)', 'NO2(GT)', 'T', 'RH',
       'AH'],
      dtype='object')


In [29]:
renamed_columns = ['CO', 'NMHC', 'C6H6', 'NOx', 'NO2', 'Temp', 'Rel_Hum', 'Abs_Hum']
df.columns = renamed_columns
df.head()

,CO,NMHC,C6H6,NOx,NO2,Temp,Rel_Hum,Abs_Hum
0,2.6,150,11.881723,166.0,113.0,13.60,48.875001,0.757754
1,2.0,112,9.397165,103.0,92.0,13.30,47.700000,0.725487
2,2.2,88,8.997817,131.0,114.0,11.90,53.975000,0.750239
3,2.2,80,9.228796,172.0,122.0,11.00,60.000000,0.786713
4,1.6,51,6.518224,131.0,116.0,11.15,59.575001,0.788794


#. 'Date': Date (DD/MM/YYYY) (Now dropped)


0. 'CO': True hourly averaged concentration CO in mg/m^3
1. 'NMHC': True hourly averaged overall Non Metanic HydroCarbons concentration in microg/m^3
2. 'C6H6': True hourly averaged Benzene concentration in microg/m^3 
3. 'NOx': True hourly averaged NOx concentration in ppb 
4. 'NO2': True hourly averaged NO2 concentration in microg/m^3
5. 'Temp': Temperature in Â°C
6. 'Rel_Hum': Relative Humidity (%)
7. 'Abs_Hum': AH Absolute Humidity

In [30]:
# # DO NOT RERUN/EDIT
# # This preprocessing step is done to keep all the units same in order to maintain uniformity.

# df['CO'] = df['CO'] * 1000
# df['NOx'] = df['NOx'] / 1.91

https://uk-air.defra.gov.uk/assets/documents/reports/cat06/0502160851_Conversion_Factors_Between_ppb_and.pdf

https://www2.dmu.dk/AtmosphericEnvironment/Expost/database/docs/PPM_conversion.pdf

In [31]:
df.head()

,CO,NMHC,C6H6,NOx,NO2,Temp,Rel_Hum,Abs_Hum
0,2600.0,150,11.881723,86.910995,113.0,13.60,48.875001,0.757754
1,2000.0,112,9.397165,53.926702,92.0,13.30,47.700000,0.725487
2,2200.0,88,8.997817,68.586387,114.0,11.90,53.975000,0.750239
3,2200.0,80,9.228796,90.052356,122.0,11.00,60.000000,0.786713
4,1600.0,51,6.518224,68.586387,116.0,11.15,59.575001,0.788794


In [35]:
independent_vars = ['CO', 'NMHC', 'C6H6', 'NOx', 'NO2', 'Rel_Hum', 'Abs_Hum']
predicative_var = ['Temp']

In [36]:
X = df[independent_vars]
y = df[predicative_var]

In [37]:
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.8, test_size=0.2)

In [41]:
# Fitting/Training
XGBModel = XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.05)
XGBModel.fit(train_X, train_y)

# Inference
predictions = XGBModel.predict(test_X)

# Accuracy
mse = mean_squared_error(test_y, predictions)
print(f"Mean Squared Error: {np.round(mse, 3)}")

Mean Squared Error: 0.054
